In [0]:
#init
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

In [0]:
RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_key",
    "cst_firstname": "first_name",
    "cst_lastname": "last_name",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "created_date"
}

#Reading from Bronze

In [0]:
df = spark.table('workspace.bronze.crm_cust_info')

#Data Transformations

In [0]:
df.display()
#Trim string data
#Rename column names
#Normalization for marital status, gender.

#Trimming

In [0]:
#Trimming values
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

#Normalization

In [0]:
df = (
    df.withColumn(
        "cst_marital_status",
        F.when(F.upper(F.col("cst_marital_status")) == "S", "Single")
        .when(F.upper(F.col("cst_marital_status")) == "M", "Married")
        .otherwise("n/a")
    )
    .withColumn(
        "cst_gndr",
        F.when(F.upper(F.col("cst_gndr")) == "F", "Female")
        .when(F.upper(F.col("cst_gndr")) == "M", "Male")
        .otherwise("n/a")
    )
)

#Renaming columns

In [0]:
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

#Write into Silver table

In [0]:
df.write.mode('overwrite').format('delta').saveAsTable('silver.crm_customers')